In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pysal as ps
import libpysal as lps
import matplotlib.pyplot as plt
import pointpats 
from pointpats import PointPattern
import seaborn as sns
import urllib
import os
import zipfile
import shapely

import warnings
warnings.simplefilter('ignore') 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
url  = 'https://github.com/cusp-london/Spatial-Data-Analysis/blob/master/London_Pubs.zip?raw=true'
path = os.path.join("data","London_Pubs.zip")
r    = urllib.request.urlretrieve(url, path)

z    = zipfile.ZipFile(path)
m    = z.extractall("data")

In [ ]:
url  = 'https://github.com/cusp-london/Spatial-Data-Analysis/blob/master/LSOA_IMD.zip?raw=true'
path = os.path.join("data","London_IMD.zip")
r    = urllib.request.urlretrieve(url, path)

z    = zipfile.ZipFile(path)
m    = z.extractall("data")

In [ ]:
from libpysal.cg import shapely_ext
from pointpats import window
from pointpats.window import poly_from_bbox, as_window, Window, to_ccf
import pointpats.quadrat_statistics as qs

In [ ]:
lsoas = lps.io.open ('data/LSOA_IMD/LSOA_IMD2019.shp') 
polys = [shp for shp in lsoas] 

boundary = shapely_ext.cascaded_union(polys)

In [ ]:
len(polys) 
len(points) 

In [ ]:
pp_pubs.plot(window= True, title= "London Pubs Point pattern")

In [ ]:
pp_pubs.plot(title="London Pubs")
plt.grid(False)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.savefig('foo.png', bbox_inches='tight', dpi=300)

In [ ]:
q_r_pubs = qs.QStatistic(pp_pubs,shape= "rectangle",nx = 5, ny = 5)
q_r_pubs.plot(title='Rectangle Quadrant Count') 
q_r_pubs.savefig('RECTANGLE.png',  bbox_inches='tight', dpi=300) 

In [ ]:
print('Chi-squared test statistic for the observed point pattern is: '+ str(q_r_pubs.chi2)) 
print('Degree of freedom is: '+str(q_r_pubs.df)) 
print('P-valus for Chi-squared test statistic is: '+str.format('{0:.6f}', q_r_pubs.chi2_pvalue)) #6d.p.

In [ ]:
q_h_pubs = qs.QStatistic(pp_pubs,shape= "hexagon",lh =0.08) 
q_h_pubs.plot(title='Hexagon Quadrant Count')
plt.savefig('HEXAGON.png') 


In [ ]:
print('Chi-squared test statistic for the observed point pattern is: '+ str(q_h_pubs.chi2)) 
print('Degree of freedom is: '+str(q_h_pubs.df)) 
print('P-valus for Chi-squared test statistic is: '+str.format('{0:.6f}', q_h_pubs.chi2_pvalue)) 

In [ ]:
lsoas_gdf=gpd.read_file('data/LSOA_IMD/LSOA_IMD2019.shp')
lsoas_gdf.head()

In [ ]:
lsoas_gdf = lsoas_gdf[['LSOA11CD', 'LSOA11NM', 'geometry']]
newcolumns = ['code','lsoa','geometry']
lsoas_gdf.columns = newcolumns
lsoas_gdf.head()

In [ ]:
lsoas_pubs_count = pd.DataFrame(ldn_pubs['lsoa'].value_counts().astype(int)).reset_index()
lsoas_pubs_count.columns = ['lsoa','Numbers']
lsoas_pubs_count.head()

In [ ]:
join_gdf = lsoas_gdf.merge(lsoas_pubs_count, on='lsoa')
join_gdf.to_file(driver='ESRI Shapefile', filename='data/lsoa_numbers.shp')
join_gdf.plot(column='Numbers', cmap='copper', scheme='quantiles', alpha=0.7, legend=True)

In [ ]:
join_gdf.head()

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12, 10))   
join_gdf['area']=join_gdf.geometry.area
join_gdf['density'] = join_gdf.apply(lambda row: 100000*row.Numbers / row.area, axis=1)
join_gdf.plot(ax=ax1, column='density', scheme='EqualInterval', cmap='coolwarm', legend=True, linewidth=0.1, edgecolor='black',
             legend_kwds={'title': "Densite of Pubs across London", 'loc': 'upper right'})

plt.title('Density of Pubs across London', fontsize=25)

plt.savefig('D3N51TY.png') 
ax1.axis('off')

In [ ]:
join_gdf.hist('density', color='blue', edgecolor='black', grid=False)

In [ ]:
EI = mp.EqualInterval(join_gdf['density'], k=5)

bins = list(EI.bins)
minimum = [join_gdf['density'].min()]
bins = minimum + bins
print(bins)

#ax1.axis('off')
#ax1.set_xlim(500000,596000)   #axis limits
#axins = ax1.inset_axes([0.66, 0.15, 0.3, 0.3])

join_gdf['density'].hist(bins=bins, grid=False, color='blue')

#bboroughs['gas_total'].hist(color='green', 
 #                           edgecolor='black', 
  #                          grid=False, 
   #                         bins=bins, 
    #                        ax=axins, figure=fig2)


In [ ]:
pp_pubs.plot(title="London Pubs")
plt.axis('off')
plt.xlabel("Average Pulse")
plt.ylabel("Calorie Burnage") 

plt.savefig('my_map_export.png') 

In [ ]:
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import gridplot,row,column
from bokeh.plotting import figure,show
output_notebook()

In [ ]:
from shapely.geometry import Point
pdf       = pd.read_csv('data/London_Pubs/LondonPubs.csv')
geometry = [Point(xy) for xy in zip(pdf.longitude, pdf.latitude)]
crs      = {'init': 'epsg:4326'} 
sdf      = gpd.GeoDataFrame(pdf, crs=crs, geometry=geometry)
pubdf  = sdf.to_crs({'init': 'epsg:27700'}) 
pubdf.head()

In [ ]:
print("???")

In [ ]:
sns.jointplot(x='longitude', y='latitude', data=pubdf, kind="kde", title='Pubs')
plt.suptitle("Bivariate Distribution of Pubs (London)", x=0.5, y=1.01)
plt.savefig('beep.png')
plt.show()
sns.jointplot(x='longitude', y='latitude', data=pubdf, kind="kde")

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
#cmap = sns.cubehelix_palette(as_cmap=True, dark=0, light=1, reverse=True)
sns.kdeplot(pubdf.longitude, pubdf.latitude, cmap='coolwarm', n_levels=70, shade=True, legend=True)
plt.suptitle("2D Kernel Density of Pubs (London)",x=   0.5, y=0.92)
plt.savefig('beep beep 2', bbox_inches='tight', dpi=300)
ax.legend()

plt.show()

In [ ]:
g = sns.jointplot(x='latitude', y='longitude', data=pubdf, kind="kde", color="r")
g.plot_joint(plt.scatter, c="black", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$latitude$", "$longitude$")
plt.suptitle("Bivariate Distribution of Pubs (London)", x=0.5, y=1.02)
plt.savefig('beep beep', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
geometry = [Point(xy) for xy in zip(pubdf.longitude, pubdf.latitude)]
crs      = {'init': 'epsg:4326'} 
sdf      = gpd.GeoDataFrame(pubdf, crs=crs, geometry=geometry)
sdf      = sdf.to_crs({'init': 'epsg:27700'}) 

In [ ]:
print('hello world')

In [ ]:
new_pubdf = gpd.GeoSeries(pubdf[['longitude', 'latitude']].apply(Point, axis=1), crs="+init=epsg:4326")
bbox = new_pubdf.total_bounds
titles=["Kernel Density: "+in_type[i] for i in range(4)]


pubneww = pubdf
sns.kdeplot(pubneww.longitude, pubneww.latitude, shade=True, cmap='Blues')




sns.set(style = 'whitegrid') # aesthetics
#sns.despine(left=True) # aesthetics
#sns.set_context('paper') # aesthetics
#plt.axis('equal')
#plt.show()   